### Load Modules

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pickle

### Define Content Source

In [3]:
OC_source = [
    ['hulu', 'https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Hulu'],
    ['netflix', 'https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Netflix'],
    ['amazon', 'https://en.wikipedia.org/wiki/List_of_original_programs_distributed_by_Amazon'],
    ['hbo','https://en.wikipedia.org/wiki/List_of_programs_broadcast_by_HBO']
]

### Build Scrape tools

In [4]:
def get_wiki(wiki_loc):
    # request from url, then extract text
    wiki_requested = requests.get(wiki_loc)
    wiki_extract = wiki_requested.text
    return wiki_extract

def fix_wiki(wiki_extract):    
    # Parse webpage
    # find all tables
    # identify tag names and retrieve corresponding attributes
    # collect results and remove nulls
    
    wiki_soup = BeautifulSoup(wiki_extract,'html5lib')
    wiki_tables = wiki_soup.find_all('table',{'class':'wikitable sortable'})
    row_collect = []
    try:
        for i in wiki_tables:
            table = i 
            table = table.find_all('tr')
            for j in table:
                contents = j.find_all(name = 'a', attrs={'href':re.compile("^(?!#)")})
                contents += j.find_all(name='span',attrs= {'class':'sorttext'})
                contents = [i.text for i in contents]
                row_collect +=[contents]
    except: 
        print('Errorneous')
    row_collect = [i for i in row_collect if len(i) != 0]
    return row_collect

### Execute

In [5]:
for i in OC_source:
    extract = get_wiki(i[1])
    cleanse = fix_wiki(extract)
    i += [extract, cleanse]

### Write

In [6]:
with open('../data/raw/data.pkl', 'wb') as file:
    pickle.dump(OC_source, file)